 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="#Match-ID" data-toc-modified-id="Match-ID-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Match ID</a></span></li><li><span><a href="#Extract-words" data-toc-modified-id="Extract-words-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract words</a></span></li><li><span><a href="#Running-W2V" data-toc-modified-id="Running-W2V-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running W2V</a></span></li></ul></div>

## Data Preparation

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/emails.csv")

In [3]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
print(df[1:2]['message'][1])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

## Match ID

In [14]:
import csv, sys, os, re, glob

In [68]:
def list_files(dir):
    r = []
    for f in glob.iglob('data/enron_set/**/*.txt', recursive=True):
        usable_f = f.replace('\\','/')
        if re.match(r'[\w\d/.-]*3\.[0-9]*\.[A-Z0-9]*\.txt', usable_f):
            r.append(usable_f)
    return r

In [108]:
def is_empty_email(contents):
    empty_pattern = re.compile("X-ZLID:.*(\r|\n)*\*{11}[\r\n]EDRM Enron Email", re.MULTILINE)
    if empty_pattern.search(contents):
         return True
    return False

In [109]:
def match_trec(file):
#     to_pattern = r"To:.*<(.*)>"
    from_pattern = r"From:(.*)"
    contents_pattern = re.compile("X-ZLID:.*[\r\n]((?:.|\r|\n)*)\*{11}[\r\n]EDRM Enron Email", re.MULTILINE)
    from_name = file.split('/')[-3][14:-8]
    email_id = file.split('/')[-1][:-4]
    date_pattern = r"Date: (.*)"
    with open(file, 'r') as text:
        string = text.read()
#         if not is_empty_email(string):
        try:
#             to_string = re.search(to_pattern, string).group(1)
#                 from_string = re.search(from_pattern, string).group(1)
            contents = re.search(contents_pattern, string)
            if contents:
                contents = contents.group(1).replace('\n','').strip()
            else:
                contents = ''
#                 contents = re.search(contents_pattern, string).group(1).replace('\n','').strip()
            date_string = re.search(date_pattern, string).group(1)
            return [from_name, date_string, contents, email_id]
        except:
            print('match_trec error:')
            print(file)
            print(string)
            print('-----------------')

In [28]:
working_path = 'data/enron_set/'

In [72]:
file_list = list_files(working_path)

In [73]:
file_list[:10]

['data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818886.LDZQSGT5FQLDRLO1WBHN2MK2DFSDVTACA.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818896.IVOAU4ATG11DYDPDLW5BEB0LXJEL14TGB.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818908.A0CV1HWH4CFTZMBCQWRRDZRJFIKDBFYJB.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818913.HRJLOFPA325QON43Q0PAAXW0NGI11IR2B.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818914.H0JO0RYAUKPP32YFJ0ICJQDTKFZ40POUA.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818915.DVZB12GWTWPMSUO2NGZ31PLQBTMN2I3OB.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818917.LHBZH0YQ2IA3FFF1LTS23GR1VFGSQLDLB.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000/3.818937.KUIKG5EH0XMXL4NLHHHTLNKELYHDH3FTB.txt',
 'data/enron_set/edrm-enron-v2_allen-p_xml.zip/text_000

In [110]:
from_names, dates, contents, email_ids = [], [], [], []
error_cnt = 0
no_info_cnt = 0
for f in file_list:
    trecid = f[:-4]
    try:
        info = match_trec(f)
        if info:
            from_names.append(info[0])
            dates.append(info[1])
            contents.append(info[2])
            email_ids.append(info[3])
        else:
            no_info_cnt += 1
    except UnicodeDecodeError:
        error_cnt += 1
print("files, errors, no info:",len(file_list),error_cnt, no_info_cnt)

files, errors, no info: 455449 331 0


In [112]:
contents = pd.DataFrame(
        {
            'id': email_ids,
            'sender': from_names,
            'dates': dates,
            'message': contents
        }
)

In [111]:
len(dates)

455118

In [113]:
contents

,dates,id,message,sender
0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)",3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,"Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1,"Mon, 25 Sep 2000 07:01:00 -0700 (PDT)",3.818886.LDZQSGT5FQLDRLO1WBHN2MK2DFSDVTACA,---------------------- Forwarded by Phillip K ...,allen-p
2,"Wed, 9 May 2001 17:13:00 -0700 (PDT)",3.818896.IVOAU4ATG11DYDPDLW5BEB0LXJEL14TGB,The west desk would like 2 analysts.,allen-p
3,"Mon, 11 Sep 2000 09:19:00 -0700 (PDT)",3.818908.A0CV1HWH4CFTZMBCQWRRDZRJFIKDBFYJB,,allen-p
4,"Mon, 7 May 2001 16:23:00 -0700 (PDT)",3.818913.HRJLOFPA325QON43Q0PAAXW0NGI11IR2B,---------------------- Forwarded by Phillip K ...,allen-p
5,"Mon, 30 Apr 2001 13:37:00 -0700 (PDT)",3.818914.H0JO0RYAUKPP32YFJ0ICJQDTKFZ40POUA,---------------------- Forwarded by Phillip K ...,allen-p
6,"Tue, 14 Nov 2000 06:33:00 -0800 (PST)",3.818915.DVZB12GWTWPMSUO2NGZ31PLQBTMN2I3OB,"Ina, Where can we put Barry T.?Phillip--------...",allen-p
7,"Mon, 13 Nov 2000 05:25:00 -0800 (PST)",3.818917.LHBZH0YQ2IA3FFF1LTS23GR1VFGSQLDLB,"Faith, Regarding the 2001 plan, the members of...",allen-p
8,"Tue, 26 Sep 2000 09:27:00 -0700 (PDT)",3.818937.KUIKG5EH0XMXL4NLHHHTLNKELYHDH3FTB,,allen-p
9,"Tue, 19 Sep 2000 09:35:00 -0700 (PDT)",3.818952.KHTELI4FF4CEDZSFPHHGSCEBZ4E4O4SHB,---------------------- Forwarded by Phillip K ...,allen-p


In [114]:
contents.to_csv('parsed_id_message_date_sender.csv')

In [125]:
ids = pd.read_csv('data/qrels.txt', sep=r'\s', header=None, names=['id', 'stratum', 'label'])

C:\Users\Micah\Anaconda3\envs\comps\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [126]:
ids.shape

(2056776, 3)

In [127]:
def extract_senario(string):
    return string[:3]

In [128]:
def clean_id(string):
    return string[4:]

In [129]:
ids['scenario'] = ids.id.apply(extract_senario)

In [130]:
ids['id'] = ids.id.apply(clean_id)

In [ ]:
'3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA.6'.count('.')

In [131]:
test_pd = ids

In [132]:
test_pd = test_pd[test_pd['id'].map(lambda x: x.count('.')) == 2]

In [133]:
test_pd.shape

(1366347, 4)

In [134]:
ids = test_pd

In [135]:
to_join_left = ids[['id', 'label', 'scenario']]

In [136]:
merged = pd.merge(to_join_left, contents)

In [137]:
merged.head(5)

,id,label,scenario,dates,message,sender
0,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,401,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",,buy-r
1,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,402,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",,buy-r
2,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,403,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",,buy-r
3,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,401,"Fri, 20 Oct 2000 01:00:00 -0700 (PDT)",,buy-r
4,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,402,"Fri, 20 Oct 2000 01:00:00 -0700 (PDT)",,buy-r


In [138]:
ids.loc[ids['id'] == '3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA']

,id,stratum,label,scenario
595318,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,401
1280910,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,402
1966502,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,403


In [141]:
merged.loc[merged['id'] == '3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA']

,id,label,scenario,dates,message,sender
1190649,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,401,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1190650,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,402,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1190651,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,403,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p


In [142]:
merged.to_csv('merged_id_label_senario_dates_message_sender.csv')

In [143]:
gold_standard = merged.loc[merged['label'].isin([0,1])]
print(len(gold_standard))

9708


In [144]:
first = gold_standard.loc[gold_standard['scenario'] == '401']
second = gold_standard.loc[gold_standard['scenario'] == '402']
third = gold_standard.loc[gold_standard['scenario'] == '403']
print(len(first), len(second), len(third))

3244 3205 3259


In [ ]:
%cp merged_id_label_senario_dates_message_sender.csv data/

In [ ]:
mapping_dict = dict()

In [ ]:
meaning_ids = ids.loc[ids['label'].isin([0,1])]

In [ ]:
len(meaning_ids.loc[ids['id'] == 'apple']) is 0

In [ ]:
merged.shape

## Extract words

In [ ]:
def extract_message(string):
    start = string.find('FileName:')
    sub = string[start: ].find('\n')
    return string[start: ][sub:]

In [ ]:
df['content'] = df.message.apply(extract_message)

In [ ]:
df.content.head(5)

In [ ]:
import re
def getWords(text):
    return " ".join(re.compile('\w*[A-Za-z]\w*').findall(text))

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
def token(text):
    tokenizer = TreebankWordTokenizer()
    return tokenizer.tokenize(text)

In [ ]:
df_test = df[1:50]

In [ ]:
df_test['words'] = df_test.content.apply(getWords)

In [ ]:
df_test['words'] = df_test.content.apply(token)

In [ ]:
df_test.words.head()

In [ ]:
df['words'] = df.content.apply(getWords)

In [ ]:
df.words.head()

In [ ]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [ ]:
test = df_shuffled[0:1000]

## Running W2V

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
n_features = 1000

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(test.words)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=42)

In [ ]:
lda.fit(tf)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 20

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)